In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import glob 

In [2]:
print(os.getcwd())
%cd ../../..

/Users/elizagoler/Documents/RecessionFertility/source/derived/county_gfrs
/Users/elizagoler/Documents/RecessionFertility


In [32]:
parquet_files = glob.glob("datastore/output/nvss/processed/*.parquet")
# Sort the files numerically by year extracted from the filename
parquet_files_sorted = sorted(parquet_files, key=lambda f: int(os.path.splitext(os.path.basename(f))[0]))
dfs = []
for f in parquet_files_sorted:
    year = os.path.splitext(os.path.basename(f))[0]
    df = pd.read_parquet(f)
    df['datayear'] = year
    dfs.append(df)
all_years_df = pd.concat(dfs, ignore_index=True)

In [33]:
all_years_df.head()

,datayear,stateres,cntyres,recwt,cntyrfip,ocntyfips
0,1968,01,01020,NaN,NaN,NaN
1,1968,01,01001,NaN,NaN,NaN
2,1968,01,01001,NaN,NaN,NaN
3,1968,01,01026,NaN,NaN,NaN
4,1968,01,01001,NaN,NaN,NaN


In [34]:
np.sort(all_years_df['datayear'].unique())

array(['1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
       '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003'], dtype=object)

In [36]:
years_all_nan_cntyrfip = all_years_df.groupby('datayear')['cntyrfip'].apply(lambda x: x.isna().all())
datayears_with_all_nan_cntyrfip = years_all_nan_cntyrfip[years_all_nan_cntyrfip].index.tolist()
print(f"Datayears with all cntyrfip NaN: {datayears_with_all_nan_cntyrfip}")


Datayears with all cntyrfip NaN: ['1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '2003']


** FIPS uses 999 to indicate unknown counties so 

In [ ]:
num_cntyres_ending_999 = all_years_df['cntyres'].astype(str).str.endswith('999').sum()
num_unique_cntyres_ending_999 = all_years_df.loc[all_years_df['cntyres'].astype(str).str.endswith('999'), 'cntyres'].nunique()
print(f"Number of unique cntyres values ending in '999': {num_unique_cntyres_ending_999}")

Number of unique cntyres values ending in '999': 50


In [43]:
# Show example rows where both 'cntyres' and 'cntyrfip' are populated (not null)
mask = all_years_df['cntyres'].notna() & all_years_df['cntyrfip'].notna()
both_populated = all_years_df[mask]

# Display about 70 example rows
example_rows = both_populated[['datayear', 'stateres', 'cntyres', 'cntyrfip']].head(70)
print("Example rows with both cntyres and cntyrfip populated (showing up to 70 rows):")
print(example_rows)


Example rows with both cntyres and cntyrfip populated (showing up to 70 rows):
         datayear stateres cntyres cntyrfip
32967773     1982       01   01001    01001
32967774     1982       01   01002    01003
32967775     1982       01   01002    01003
32967776     1982       01   01002    01003
32967777     1982       01   01002    01003
...           ...      ...     ...      ...
32967838     1982       01   01008    01015
32967839     1982       01   01008    01015
32967840     1982       01   01008    01015
32967841     1982       01   01008    01015
32967842     1982       01   01008    01015

[70 rows x 4 columns]


In [52]:
nchs_fips_crosswalk = pd.read_csv('datastore/raw/crosswalks/nchs_fips_crosswalk.csv')

In [53]:
nchs_fips_crosswalk.head()

,nchs_state,stateoc,nchs_county,countyoc,nchs_msa,metro,fipsst,fips_msa,countyname,statename,fipsco
0,1,1,1,1,196,1,1,5240,Autauga,Alabama,1001
1,1,1,2,2,192,1,1,5160,Baldwin,Alabama,1003
2,1,1,3,3,0,2,1,0,Barbour,Alabama,1005
3,1,1,4,4,0,2,1,0,Bibb,Alabama,1007
4,1,1,5,5,37,1,1,1000,Blount,Alabama,1009


In [54]:
nchs_fips_crosswalk = nchs_fips_crosswalk.rename(columns={
    'nchs_state': 'stateres',
    'nchs_county': 'cntyres'
})

In [55]:
nchs_fips_crosswalk

,stateres,stateoc,cntyres,countyoc,nchs_msa,metro,fipsst,fips_msa,countyname,statename,fipsco
0,1,1,1,1,196,1,1,5240,Autauga,Alabama,1001
1,1,1,2,2,192,1,1,5160,Baldwin,Alabama,1003
2,1,1,3,3,0,2,1,0,Barbour,Alabama,1005
3,1,1,4,4,0,2,1,0,Bibb,Alabama,1007
4,1,1,5,5,37,1,1,1000,Blount,Alabama,1009
...,...,...,...,...,...,...,...,...,...,...,...
3127,51,51,19,19,0,2,56,0,Sweetwater,Wyoming,56037
3128,51,51,20,20,0,2,56,0,Teton,Wyoming,56039
3129,51,51,21,21,0,2,56,0,Uinta,Wyoming,56041
3130,51,51,22,22,0,2,56,0,Washakie,Wyoming,56043


In [ ]:
all_years_df.dtypes